In [23]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb
%matplotlib inline

# Data files full clean

In [24]:
import os
import re

if not os.path.exists("./cleandata"):
    os.makedirs("./cleandata")
content = os.listdir("./newdata")

for item in content:
    # delete the text files and the urls files to clean the library
    print(item)
    if item.find(".txt") > 0:
        # print(item)
        with open("./newdata/{}".format(item), "r", encoding="iso-8859-1") as f:
            lines = f.readlines()
        # if (os.path.exists("./cleandata/{}".format(item))):
        with open("./cleandata/{}".format(item), "w+") as f:

            for line in lines:
                if "10 Messungen" in line.strip("\n"):
                    line = line[line.find("Ch") :]
                    # print(newline)
                # if (line.strip("\n") != "þStartfischer1 p33-6 hf8 fc6x 0.9km")
                if "Ch" in line.strip("\n") or "V" in line.strip("\n"):
                    f.write(line.replace(";", ""))
f.close()

# print(content)

fischer1_p33_6_hf8_fc6x_0_9km.txt
fischer1_p33_6_hf8_fc6x_4km.txt
fischer2_p33_6_hf8_fc6x.txt
fischer2_p33_6_hf8_fc6x_0_9km.txt
fischer2_p33_6_hf8_fc6x_4km.txt
fischer_p33_6_hf8_fc6x.txt
madshus_base7_ts_blue.txt
madshus_base7_ts_blue_ clockwise 45 turn.txt
madshus_base7_ts_blue_point.txt


In [90]:
a={}
lst={}
content = os.listdir("./cleandata")
result=pd.DataFrame()
verhaeltnis=pd.DataFrame()
deltas=pd.DataFrame()
for x in content:
    if (x.find(".txt") > 0 and x != ".git"):
        #print(x)
        df = pd.read_csv("./cleandata/{}".format(x),names=["channal", "min", "max", "x", "y"],sep="\s+",encoding="ISO-8859-1")
        dx = df.drop(["x", "y"], axis=1)
        dx["min"] = dx["min"]  # .map(lambda x: x.lstrip("+-;").rstrip("aAbBcC;"))
        dx["channal"] = dx["channal"]  # .map(lambda x: x.lstrip(";").rstrip(";"))
        if 100 in dx.index:
            dx = dx.drop(100)
        dx["min"] = pd.to_numeric(dx["min"])
        dx["max"] = pd.to_numeric(dx["max"])
        dx["deltas"] = dx["max"] - dx["min"]
        ########### Extract channals ######

        ch0=dx[dx.channal == "Ch0:"]
        ch1=dx[dx.channal == "Ch1:"]
        ch2=dx[dx.channal == "Ch2:"]
        ch3=dx[dx.channal == "Ch3:"]
        ch4=dx[dx.channal == "Ch4:"]
        ch5=dx[dx.channal == "Ch5:"]
        ch6=dx[dx.channal == "Ch6:"]
        ch7=dx[dx.channal == "Ch7:"]
        ch8=dx[dx.channal == "Ch8:"]
        #print(ch0.head())
        ############ build the deltas #############
        delta0= ch0["max"] -  ch0["min"]
        lst['delta0']=delta0
        delta1= ch1["max"] -  ch1["min"]
        lst['delta1']=delta1
        delta2= ch2["max"] -  ch2["min"]
        lst['delta2']=delta2
        delta3= ch3["max"] -  ch3["min"]
        lst['delta3']=delta3
        delta4= ch4["max"] -  ch4["min"]
        lst['delta4']=delta4
        delta5= ch5["max"] -  ch5["min"]
        lst['delta5']=delta5
        delta6= ch6["max"] -  ch6["min"]
        lst['delta6']=delta6
        delta7= ch7["max"] -  ch7["min"]
        lst['delta7']=delta7
        delta8= ch8["max"] -  ch8["min"]
        lst['delta8']=delta8
        #print(delta0.head())
        ##################### format reset index

        df_new = pd.DataFrame.from_dict(lst)
        df_new=df_new.apply(lambda x: pd.Series(x.dropna().values))
        #print(df_new.head(3))
        df_new.to_excel("dfnew.xlsx")
        dv=dx[dx.channal == "V:"]

        ########## connect the name of Measurement
        dv.index.name=x.strip('.txt')
        dx.index.name = x.strip('.txt')
        df_new.index.name=x.strip('.txt')

        dx=dx.reset_index()
        dv=dv.reset_index()
        df_new=df_new.reset_index()

        if not (dx.empty):
            a[x.strip('.txt')]=dx
        dv=dv.replace('V:', 'V:{}'.format(x.strip('.txt')))
        #drop empty Dataframes
        if not (dx.empty):
            result=pd.concat([result,dx.append(pd.Series(name='Verh..'))],axis=1)
            verhaeltnis=pd.concat([verhaeltnis,dv],axis=1)
            new_df=result.append(verhaeltnis)
            deltas=pd.concat([deltas,df_new],axis=1)

#new_df=pd.concat([result,verhaeltnis])
#print(result.head())
result.to_excel("result.xlsx")
verhaeltnis=verhaeltnis.drop(["max","deltas"], axis=1)
verhaeltnis=verhaeltnis.rename(columns={"min": "V"})
verhaeltnis.head()
verhaeltnis.to_excel("verhaeltnis.xlsx")
new_df.to_excel("FullData.xlsx")
deltas.to_excel("deltas.xlsx")